In [21]:
import morphs
import behav
from behav import loading
import pandas as pd

In [9]:
data_folder = '/mnt/cube/RawData/Zog/'
behav_data = loading.load_data_pandas(morphs.subj.BEHAVE_SUBJS, data_folder)

/usr/local/anaconda/envs/morphs/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2818: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/anaconda/envs/morphs/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2818: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [175]:
def parse_behav_data_stim_id(df):
    df = df[(df['response'] != 'none') & (df['type_'] == 'normal')]
    df['stim_id'] = df['stimulus'].str.split('/').str[-1].str[:-4]
    df = df[df['stim_id'].str.len() == 5]
    df = df[df['stim_id'].str[1:] != '_rec']
    df['subj'] = subj
    morphs.data.parse.stim_id(df)
    return df

In [177]:
cumulative_data = pd.concat([parse_behav_data_stim_id(behav_data[subj]) for subj in morphs.subj.BEHAVE_SUBJS])

/usr/local/anaconda/envs/morphs/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [178]:
def parse_behav_data_inverted(df):
    inverted_map = df[(df['morph_pos']==1)].groupby(['subj', 'morph_dim']).agg(lambda x: x.iloc[0])['class_'] == 'R'
    df = df.join(inverted_map.to_frame(name='inverted'), on=('subj', 'morph_dim'), how='left', sort=False)
    df['greater_response'] = (df['response'] == 'R') != (df['inverted'])
    return df

In [181]:
cumulative_data = parse_behav_data_inverted(cumulative_data)

In [26]:
cumulative_data.groupby('subj').agg('count')['index']

subj
B1082    1280159
B1088     545599
B1101     321631
B1105    1485138
B1107     111549
B1218     449423
B1222    1248989
B979       68701
Name: index, dtype: int64

In [160]:
del cumulative_data['inverted']
del cumulative_data['greater_response']

In [113]:
def is_inverted(subj, morph_dim):
    left, right = morphs.subj.TRAINING[subj].lower().split('|')
    les, gre = morph_dim
    assert (les in left) != (gre in left), (row, morphs.subj.TRAINING[row['subj']])
    assert (les in right) != (gre in right), (row, morphs.subj.TRAINING[row['subj']])
    return gre in left

In [183]:
for (subj, morph_dim), group in cumulative_data.groupby(['subj', 'morph_dim']):
    cumulative_data.loc[group.index, 'inverted_slow'] = is_inverted(subj, morph_dim)

In [186]:
assert np.all(cumulative_data['inverted'] == cumulative_data['inverted_slow'])

In [37]:
import numpy as np
import scipy.optimize as op

eta_bounds = 1e-16

def four_param_logistic(p):
    """4p logistic function maker.
    
    Returns a function that accepts x and returns y for
    the 4-parameter logistic defined by p.
    
    The 4p logistic is defined by:
    y = A + (K - A) / (1 + exp(-B*(x-M)))
    
    Args:
        p: an iterable of length 4
            A, K, B, M = p
    
    Returns:
        A function that accepts a numpy array as an argument 
        for x values and returns the y values for the defined 4pl curve.
            
    Marvin Thielk 2016
    mthielk@ucsd.edu
    """
    A, K, B, M = p
    def f(x):
        return A + (K - A) / (1 + np.exp(-B*(x-M)))
    return f

def ln_like(p, x, y):
    """log likelihood for fitting the four parameter logistic.
    
    Args:
        p: an iterable of length 4
            A, K, B, M = p
        x: a numpy array of length n
        y: a numpy array of length n
            must be of dtype double or float so multiplication works
    
    Returns:
        The log-likelihood that the samples y are drawn from a distribution
        where the 4pl(x; p) is the probability of getting y=1
            
    Marvin Thielk 2016
    mthielk@ucsd.edu
    """
    p_4pl = four_param_logistic(p)
    probs = p_4pl(x)
    return np.sum(y * np.log(probs) + (1 - y) * np.log(1 - probs))

def dln_like(p, x, y):
    """gradient of the log likelihood for fitting the four parameter logistic.
    
    Args:
        p: an iterable of length 4
            A, K, B, M = p
        x: a numpy array of length n
        y: a numpy array of length n
            must be of dtype double or float so multiplication works
    
    Returns:
        The gradient of the log-likelihood that the samples y are drawn from 
        a distribution where the 4pl(x; p) is the probability of getting y=1
            
    Marvin Thielk 2016
    mthielk@ucsd.edu
    """
    A, K, B, M = p
    def f(x):
        return A + (K - A) / (1 + np.exp(-B*(x-M)))
    def df(x):
        temp1 = np.exp(-B*(x-M))
        dK = 1. / (1. + temp1)
        dA = 1. - dK
        temp2 = temp1 / (1. + temp1) ** 2
        dB = (K - A) * (x - M) * temp2
        dM = -(K - A) * B * temp2
        return np.vstack((dA, dK, dB, dM))
    p_4pl = f(x)
    d_p_4pl = df(x)
    return np.sum(y * d_p_4pl / (p_4pl) - (1 - y) * d_p_4pl / (1 - p_4pl), 1)

def nll(*args):
    """negative log-likelihood for fitting the 4 param logistic."""
    return -ln_like(*args)

def ndll(*args):
    """negative grad of the log-likelihood for fitting the 4 param logistic."""
    return -dln_like(*args)

def est_pstart(x, y):
    """basic estimation of a good place to start log likelihood maximization.
    
    Args:
        x: a numpy array of length n
            assumes a finite number of unique x values
        y: a numpy array of length n
            must be of dtype double or float so multiplication works
    
    Returns:
        p_start: an iterable of length 4 that should be a reasonable spot to
            start the optimization
            A, K, B, M = p_start
            
    Marvin Thielk 2016
    mthielk@ucsd.edu
    """
    p_start = [.01, .99, .2, 0]
    x_vals = np.unique(x)
    p_start[3] = np.mean(x_vals)
    y_est = np.array([np.mean(y[x==i]) for i in x_vals])
    midpoint_est = np.mean(np.where((y_est[0:-1]<.5) & (y_est[1:]>=.5)))
    if np.isnan(midpoint_est):
        return p_start
    p_start[3] = midpoint_est
    return p_start

def fit_4pl(x, y, p_start=None, verbose=False):
    """Fits a 4 parameter logistic function to the data.
    
    Args:
        x: a numpy array of length n
            assumes a finite number of unique x values
        y: a numpy array of length n
            must be of dtype double or float so multiplication works
    optional:
        p_start: an iterable of length 4 that would be a reasonable spot to
            start the optimization. If None, tries to estimate it.
            A, K, B, M = p_start
            default=None
        verbose: boolean flag that allows printing of more error messages.
    
    Returns:
        p_result: an iterable of length 4 that defines the model that 
        is maximally likely
            A, K, B, M = p_result
            
    Marvin Thielk 2016
    mthielk@ucsd.edu
    """
    try:
        if not p_start:
            p_start = est_pstart(x, y)
    except TypeError:
        pass
    for i in range(3):
        if verbose and i > 0:
            print 'retry', i
        result = op.minimize(nll, p_start, args=(x, y), jac=ndll, bounds=((eta_bounds,1-eta_bounds), (eta_bounds,1-eta_bounds), (None, None), (None, None)))
        if result.success:
            return result.x
        else:
            if verbose:
                print p_start, 'failure', result
            p_start = result.x
    return False

In [56]:
psychometric_params = {}
for subj, subj_group in cumulative_data.groupby('subj'):
    psychometric_params[subj] = {}
    for dim, dim_group in subj_group.groupby('morph_dim'):
        x = dim_group['morph_pos'].astype(float).values
        y = dim_group['greater_response'].astype(float).values
        psychometric_params[subj][dim] = fit_4pl(x, y, verbose=True)